In [1]:
import pickle
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from geopy.distance import geodesic
import geopy.geocoders
from IPython.core.display import display, HTML

gl = geopy.geocoders.Nominatim(user_agent="geoapi")

region={
      "DDA Market": "Delhi",
    "Ghazipur Market": "Delhi",
    "Barnala Market": "Punjab",
    "Ludhiana Market": "Punjab",
    "Sunam Dhuri Market": "Punjab",
    "Amritsar Market": "Punjab",
    "Faridabad Market": "Haryana",
    "Yamuna Nagar Market": "Haryana"
}

market_mapping = {
    "DDA Market": "MarketA",
    "Ghazipur Market": "MarketB",
    "Barnala Market": "MarketC",
    "Ludhiana Market": "MarketD",
    "Sunam Dhuri Market": "MarketE",
    "Amritsar Market": "MarketF",
    "Faridabad Market": "MarketG",
    "Yamuna Nagar Market": "MarketH"
}


market_locations = {
    "DDA Market": (28.6333, 77.0681),
    "Ghazipur Market": (28.6288, 77.3280),
    "Barnala Market": (31.1636, 74.9423),
    "Ludhiana Market": (30.8895, 75.8622),
    "Sunam Dhuri Market": (30.1310, 75.8006),
    "Amritsar Market": (31.6002, 74.8880),
    "Faridabad Market": (28.3745, 77.2965),
    "Yamuna Nagar Market": (30.1282, 77.2764)
}

def get_distance(user_location, market_location):
    """Calculate distance between user and market."""
    return geodesic(user_location, market_location).km

def get_coordinates(location_name):
    """Convert location name to latitude and longitude."""
    try:
        location = gl.geocode(location_name)
        return (location.latitude, location.longitude) if location else None
    except:
        return None

def make_clickable(val):
    """Convert URL string into a clickable link"""
    return f'<a href="{val}" target="_blank">Route</a>' if "http" in val else val

def compare_markets(fore_date, fish, size, user_location):
    if fore_date < datetime.strptime("07-03-2025", "%d-%m-%Y"):
        return "Enter a date after 7th March 2025 only"

    user_coords = get_coordinates(user_location)
    if not user_coords:
        return "Invalid location. Please enter a valid place."

    market_predictions = []

    train_data = pd.read_excel("north_india_agg_27march.xlsx")
    train_data.dropna(subset=['Date'], inplace=True)
    train_data['Date'] = pd.to_datetime(train_data['Date'], format='%d-%m-%Y', errors='coerce')
    full_dates = pd.DataFrame({'Date': pd.date_range(start=train_data['Date'].min(), end=train_data['Date'].max(), freq='D')})
    train_data = full_dates.merge(train_data, on='Date', how='left')
    train_data.set_index('Date', inplace=True)
    train_data.replace(0, np.nan, inplace=True)
    train_data = train_data.interpolate(method="polynomial", order=2)
    train_data = train_data.interpolate(method="linear")
    train_data = train_data.ffill().bfill()


    scaler = StandardScaler()
    train_scaled = pd.DataFrame(scaler.fit_transform(np.cbrt(train_data)),
                                columns=train_data.columns,
                                index=train_data.index)


    with open('model.pkl', 'rb') as file:
        model = pickle.load(file)

    baseline_date = datetime.strptime("07-03-2025", "%d-%m-%Y")
    steps = (fore_date - baseline_date).days
    lag_order = model.k_ar
    forecast_scaled = model.forecast(train_scaled.values[-lag_order:], steps=steps)
    forecast = scaler.inverse_transform(forecast_scaled)
    forecast_data = pd.DataFrame(forecast, columns=train_data.columns)
    forecast_data = np.power(forecast_data, 3)


    for market, coords in market_locations.items():
        fish_and_size = f"{fish}_{size}_{market_mapping[market]}"
        if fish_and_size in forecast_data.columns:
            predicted_price = np.round(forecast_data.loc[0, fish_and_size])
            distance = get_distance(user_coords, coords)
            maps_url = f"https://www.google.com/maps/dir/{user_coords[0]},{user_coords[1]}/{coords[0]},{coords[1]}"
            market_predictions.append({
                "Region": region[market],
                "Market": f"{market}",
                "Prediction in ₹": predicted_price,
                "Distance in km": round(distance, 2),
                "Route": maps_url
            })
        else:
            market_predictions.append({
                "Region": region[market],
                "Market": f"{market}",
                "Prediction in ₹": "Not Sold Here",
                "Distance in km": round(get_distance(user_coords, coords), 2),
                "Route": "N/A"
            })


    market_predictions.sort(key=lambda x: (x["Prediction in ₹"] if isinstance(x["Prediction in ₹"], (int, float)) else -1), reverse=True)


    df = pd.DataFrame(market_predictions)
    df["Route"] = df["Route"].apply(make_clickable)
    display(HTML(df.to_html(escape=False)))



In [2]:
compare_markets(datetime.strptime("30-06-2025", "%d-%m-%Y"),"Catla","Small","Karol Bagh, Delhi")

,Region,Market,Prediction in ₹,Distance in km,Route
0,Delhi,DDA Market,216.0,12.02,Route
1,Delhi,Ghazipur Market,206.0,13.85,Route
2,Punjab,Barnala Market,180.0,352.87,Route
3,Punjab,Ludhiana Market,150.0,279.14,Route
4,Punjab,Amritsar Market,150.0,394.82,Route
5,Haryana,Yamuna Nagar Market,85.0,163.73,Route
6,Punjab,Sunam Dhuri Market,Not Sold Here,212.13,N/A
7,Haryana,Faridabad Market,Not Sold Here,32.61,N/A


In [3]:
compare_markets(datetime.strptime("01-05-2025", "%d-%m-%Y"),"Rohu","Medium","gurugram")

,Region,Market,Prediction in ₹,Distance in km,Route
0,Delhi,DDA Market,302.0,19.06,Route
1,Delhi,Ghazipur Market,242.0,34.38,Route
2,Punjab,Sunam Dhuri Market,207.0,219.95,Route
3,Punjab,Barnala Market,200.0,360.86,Route
4,Punjab,Ludhiana Market,200.0,291.58,Route
5,Haryana,Faridabad Market,124.0,27.96,Route
6,Punjab,Amritsar Market,Not Sold Here,404.33,N/A
7,Haryana,Yamuna Nagar Market,Not Sold Here,185.94,N/A
